# CV with CNN | Project 1
The case study is from a dataset from Kaggle. 

Link to the Kaggle project site:

https://www.kaggle.com/c/plant-seedlings-classification

The dataset has to be downloaded from the above Kaggle web site.

Can you differentiate a weed from a crop seedling?

The ability to do so effectively can mean better crop yields and better stewardship of the environment.

The Aarhus University Signal Processing group, in collaboration with the University of Southern
Denmark, has recently released a dataset containing images of approximately 960 unique plants
belonging to 12 species at several growth stages.

The points distribution for this case is as follows:
1. Read the images and generate the train and test dataset (5 points)
2. Divide the data set into Train and validation data sets
3. Initialize & build the model (10 points)
4. Optimize the model (8 points)
5. Predict the accuracy for both train and validation data (7 points)

# 1. Read the images and generate the train and test dataset
# 2. Divide the data set into Train and validation data sets



In [1]:
#For ease of working with large Dataset, we have unzipped & uploaded it to Google Drive
#So, first let's mount the drive

#Importing drive module from google.colab library
from google.colab import drive

#Mount the drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importing tensorflow library
import tensorflow as tf

#ImageDataGenerator declaration with 20% data as validation (80% for training)
train_img_generator= tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

#ImageDataGenerator declaration for Test dataset
test_img_generator= tf.keras.preprocessing.image.ImageDataGenerator()

In [3]:
#Build training generator. 
train_generator = train_img_generator.flow_from_directory('drive/My Drive/plant-seedlings-classification/train',
                                                    target_size=(64, 64),
                                                    subset='training',
                                                    batch_size=64)

#Build validation generator
val_generator = train_img_generator.flow_from_directory('drive/My Drive/plant-seedlings-classification/train',
                                                   target_size=(64, 64),                                                   
                                                   subset='validation',
                                                   batch_size=64)

#Build test generator
test_generator = test_img_generator.flow_from_directory('drive/My Drive/plant-seedlings-classification',
                                                   classes=['test'],
                                                   target_size=(64, 64),                                                   
                                                   batch_size=64)

Found 3803 images belonging to 12 classes.
Found 947 images belonging to 12 classes.
Found 794 images belonging to 1 classes.


# 3. Initialize & build the model

In [4]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(64,64,3)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(16, kernel_size=(5,5), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5,5), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 60, 60, 16)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        12832     
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0

In [7]:
#Fitting the model with Training dataset & validating the same with validation dataset
model.fit_generator(train_generator,
                    epochs=20,
                    steps_per_epoch= 3803//64,  #Number of training images//batch_size
                    validation_data=val_generator,
                    validation_steps = 947//64 #Number of validation images//batch_size
                   )

Epoch 1/20
59/59 [==============================] - 56s 948ms/step - loss: 2.7314 - acc: 0.3057 - val_loss: 2.5180 - val_acc: 0.2634
Epoch 2/20
59/59 [==============================] - 55s 939ms/step - loss: 1.4707 - acc: 0.5154 - val_loss: 21.6079 - val_acc: 0.1306
Epoch 3/20
59/59 [==============================] - 55s 936ms/step - loss: 1.2126 - acc: 0.5988 - val_loss: 26.1571 - val_acc: 0.2221
Epoch 4/20
59/59 [==============================] - 54s 923ms/step - loss: 0.9071 - acc: 0.6918 - val_loss: 13.3600 - val_acc: 0.3214
Epoch 5/20
59/59 [==============================] - 54s 912ms/step - loss: 0.7461 - acc: 0.7507 - val_loss: 4.3008 - val_acc: 0.4933
Epoch 6/20
59/59 [==============================] - 53s 897ms/step - loss: 0.6173 - acc: 0.7871 - val_loss: 1.9040 - val_acc: 0.6228
Epoch 7/20
59/59 [==============================] - 53s 897ms/step - loss: 0.5549 - acc: 0.8082 - val_loss: 1.3960 - val_acc: 0.6853
Epoch 8/20
59/59 [==============================] - 52s 889ms/step

# 4. Optimize the model

In [8]:
#With the above model, we saw that the Model has started overfitting on the Training dataset
#To avoid this, we will use Image augmentation so that model learns on augmented image each time which will ensure that it doesn't overfit

#ImageDataGenerator declaration with Image augmentation that will help avoid over-fitting of Training dataset
train_img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                     width_shift_range=0.2,
                                                                     height_shift_range=0.2,
                                                                     horizontal_flip=True,
                                                                     validation_split=0.2
                                                                    )

In [9]:
#Build training generator. 
train_generator = train_img_generator.flow_from_directory('drive/My Drive/plant-seedlings-classification/train',
                                                    target_size=(64, 64),
                                                    subset='training',
                                                    batch_size=64)

#Build validation generator
val_generator = train_img_generator.flow_from_directory('drive/My Drive/plant-seedlings-classification/train',
                                                   target_size=(64, 64),                                                   
                                                   subset='validation',
                                                   batch_size=64)

Found 3803 images belonging to 12 classes.
Found 947 images belonging to 12 classes.


In [10]:
#Reinitialising the model without saving the learnt weights so that it starts from scratch
json_string = model.to_json()
model = tf.keras.models.model_from_json(json_string)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
#Fitting the model on Augmented Training dataset
model.fit_generator(train_generator,
                    epochs=100,
                    steps_per_epoch= 3803//64,  #Number of training images//batch_size
                    validation_data=val_generator,
                    validation_steps = 947//64 #Number of test images//batch_size
                   )

Epoch 1/100
59/59 [==============================] - 60s 1s/step - loss: 2.9677 - acc: 0.2364 - val_loss: 2.4792 - val_acc: 0.1116
Epoch 2/100
59/59 [==============================] - 60s 1s/step - loss: 2.0447 - acc: 0.3300 - val_loss: 3.7391 - val_acc: 0.2667
Epoch 3/100
59/59 [==============================] - 59s 1s/step - loss: 1.8672 - acc: 0.3825 - val_loss: 12.0739 - val_acc: 0.1752
Epoch 4/100
59/59 [==============================] - 58s 986ms/step - loss: 1.7487 - acc: 0.4164 - val_loss: 5.3540 - val_acc: 0.2879
Epoch 5/100
59/59 [==============================] - 58s 985ms/step - loss: 1.6220 - acc: 0.4447 - val_loss: 2.3023 - val_acc: 0.4587
Epoch 6/100
59/59 [==============================] - 57s 964ms/step - loss: 1.5732 - acc: 0.4773 - val_loss: 1.5203 - val_acc: 0.5279
Epoch 7/100
59/59 [==============================] - 57s 973ms/step - loss: 1.5361 - acc: 0.4643 - val_loss: 1.5157 - val_acc: 0.5413
Epoch 8/100
59/59 [==============================] - 57s 968ms/step - 

In [13]:
#The above model got rid of over-fitting as most of the times validation accuracy was better than training accuracy
#But, as we can see the accuracy for both training dataset & validation dataset has started saturating between 85% and 90%
#Let's try to add couple of more CNN layers & see how the model performs

#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(64,64,3)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(16, kernel_size=(5,5), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5,5), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(12, activation='softmax'))

In [14]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 60, 60, 16)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        12832     
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 54, 54, 32)        9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 54, 54, 32)        1

In [16]:
#Fitting the model with Training dataset & validating the same with validation dataset
model.fit_generator(train_generator,
                    epochs=100,
                    steps_per_epoch= 3803//64,  #Number of training images//batch_size
                    validation_data=val_generator,
                    validation_steps = 947//64 #Number of validation images//batch_size
                   )

Epoch 1/100
59/59 [==============================] - 60s 1s/step - loss: 2.4176 - acc: 0.2894 - val_loss: 7.2331 - val_acc: 0.1395
Epoch 2/100
59/59 [==============================] - 60s 1s/step - loss: 1.7492 - acc: 0.4226 - val_loss: 3.0907 - val_acc: 0.2422
Epoch 3/100
59/59 [==============================] - 59s 1s/step - loss: 1.5640 - acc: 0.4790 - val_loss: 2.5913 - val_acc: 0.3516
Epoch 4/100
59/59 [==============================] - 59s 1s/step - loss: 1.4602 - acc: 0.5205 - val_loss: 2.4663 - val_acc: 0.4029
Epoch 5/100
59/59 [==============================] - 58s 987ms/step - loss: 1.3422 - acc: 0.5389 - val_loss: 1.2670 - val_acc: 0.5748
Epoch 6/100
59/59 [==============================] - 58s 987ms/step - loss: 1.2424 - acc: 0.5841 - val_loss: 1.1850 - val_acc: 0.6239
Epoch 7/100
59/59 [==============================] - 58s 979ms/step - loss: 1.1244 - acc: 0.6245 - val_loss: 1.1241 - val_acc: 0.6417
Epoch 8/100
59/59 [==============================] - 57s 968ms/step - loss

With this model, we got an accuracy of over 90% after 100 Epochs after adding 2 more CNN layers.

We can train the model for few more Epochs as the model's accuracy is slowly increasing or else we can try to add more CNN layers / fully connected layers to improve the model's accuracy further.

We can keep trying until we are satisfied with the results given the limited resources we have or when we don't see any further scope to improve the model's accuracy any further.

# 5. Predict the accuracy for both train and validation data

In [17]:
#Using evaluate_generator on the ImageDataGenerator to predict the accuracy on training & validation Datasets
train_loss, train_acc = model.evaluate_generator(train_generator, 3803//64)
val_loss, val_acc = model.evaluate_generator(val_generator, 947//64)
print('Training accuracy is', train_acc * 100, '%')
print('Validation accuracy is', val_acc * 100, '%')

Training accuracy is 93.37923526763916 %
Validation accuracy is 90.51339030265808 %
